In [1]:
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import glob
from tqdm import tqdm

## NASA FIRMS - Fire Information for Resource Management System

FIRMS distributes Near Real-Time (NRT) active fire data within 3 hours of satellite observation from the Moderate Resolution Imaging Spectroradiometer ([MODIS](https://modis.gsfc.nasa.gov/)) aboard the Aqua and Terra satellites, and the Visible Infrared Imaging Radiometer Suite ([VIIRS](https://www.jpss.noaa.gov/viirs.html)) aboard S-NPP and NOAA 20.


### Acknowledgement & Disclaimer

We acknowledge the use of data and/or imagery from NASA's FIRMS (https://earthdata.nasa.gov/firms), part of NASA's Earth Observing System Data and Information System (EOSDIS).

* Do not use for the preservation of life or property. Satellite-derived active fire / thermal anomalies have limited accuracy.
* Active fire/thermal anomalies may be from fire, hot smoke, agriculture or other sources.
* Cloud cover may obscure active fire detections.

Please see the [official page](https://earthdata.nasa.gov/earth-observation-data/near-real-time/citation#ed-lance-disclaimer) for further details.

## Defining the Target Zone
Define the latitude & longitude ranges of your target country, zone, etc. The example is for Turkey.

In [2]:
LAT_RANGE = (36, 42)
LON_RANGE = (26, 45)

### Read and check the chunks

The archive fire/hotspot datasets could be requested at 
https://firms.modaps.eosdis.nasa.gov/download/ in yearly chunks for each instrument.

* MODIS Collection 6.1: Temporal Coverage: 11 November 2000 - present
* VIIRS S-NPP 375m: Temporal Coverage: 20 January 2012 - present
* VIIRS NOAA-20 375m: Temporal Coverage: 1 January 2020 - present

Since NOAA-20 has less than 2 years data let's focus ont the other instruments.

In [3]:
filenames = glob.glob('../data/raw_data/*/*.csv')
filenames

['../data/raw_data/DL_FIRE_M-C61_216014/fire_archive_M-C61_216014.csv',
 '../data/raw_data/DL_FIRE_M-C61_216003/fire_archive_M-C61_216003.csv',
 '../data/raw_data/DL_FIRE_SV-C2_216019/fire_archive_SV-C2_216019.csv',
 '../data/raw_data/DL_FIRE_M-C61_216012/fire_archive_M-C61_216012.csv',
 '../data/raw_data/DL_FIRE_SV-C2_216899/fire_archive_SV-C2_216899.csv',
 '../data/raw_data/DL_FIRE_M-C61_216898/fire_archive_M-C61_216898.csv',
 '../data/raw_data/DL_FIRE_M-C61_216006/fire_archive_M-C61_216006.csv',
 '../data/raw_data/DL_FIRE_SV-C2_216007/fire_archive_SV-C2_216007.csv',
 '../data/raw_data/DL_FIRE_M-C61_216010/fire_archive_M-C61_216010.csv',
 '../data/raw_data/DL_FIRE_SV-C2_216015/fire_archive_SV-C2_216015.csv',
 '../data/raw_data/DL_FIRE_SV-C2_216011/fire_archive_SV-C2_216011.csv',
 '../data/raw_data/DL_FIRE_SV-C2_230336/fire_archive_SV-C2_230336.csv',
 '../data/raw_data/DL_FIRE_SV-C2_230336/fire_nrt_SV-C2_230336.csv',
 '../data/raw_data/DL_FIRE_M-C61_216016/fire_archive_M-C61_216016.cs

# Process each chunk

We removed fire readings with low or less than 50 confidence. We also took only the zones that falls within our target latitude & longitude ranges. For simplicity the coordinates are rounded to two decimal degrees. That is roughly 1.1 km at the Equator. For better spatial resolution the original VIIRS records could be used.


In [4]:
chunks = []
cols_to_read = ['latitude', 'longitude', 'acq_date', 'satellite', 'instrument', 'confidence']
for f in tqdm(filenames):
    fire = pd.read_csv(f, usecols=cols_to_read, parse_dates=['acq_date'], low_memory=False)
    if fire.satellite.loc[0] in ['Terra', 'Aqua', 'N']:
        try:
            fire = fire[(fire.latitude >= LAT_RANGE[0]) &\
                              (fire.latitude <= LAT_RANGE[1]) &\
                              (fire.longitude >= LON_RANGE[0]) &\
                              (fire.longitude <= LON_RANGE[1])]
            fire.latitude = fire.latitude.round(2)
            fire.longitude = fire.longitude.round(2)
            fire.confidence = fire.confidence.replace({'l': 0, 'n': 50, 'h': 100})
            daily_fires = fire.groupby(
                ['latitude', 'longitude', 'acq_date', 'satellite', 'instrument']).confidence.max().reset_index()
            daily_fires = daily_fires[daily_fires.confidence >= 50]  # Remove low confidence records
            chunks.append(daily_fires)
        except:
            pass
    else:
        pass

100%|███████████████████████████████████████████| 21/21 [02:55<00:00,  8.34s/it]


In [5]:
daily_fires.head()

,latitude,longitude,acq_date,satellite,instrument,confidence
0,36.0,36.68,2015-05-28,N,VIIRS,50
1,36.0,36.69,2015-05-28,N,VIIRS,50
2,36.0,36.74,2015-08-10,N,VIIRS,50
3,36.0,36.75,2015-07-23,N,VIIRS,50
4,36.0,36.80,2015-08-07,N,VIIRS,50


We are merging the chunks we created into one full dataframe.

In [6]:
full_dataset = pd.concat(chunks)
full_dataset

,latitude,longitude,acq_date,satellite,instrument,confidence
0,36.00,43.38,2016-11-25,Aqua,MODIS,62
1,36.00,43.42,2016-11-13,Terra,MODIS,71
2,36.00,43.89,2016-06-10,Terra,MODIS,100
3,36.01,36.34,2016-09-20,Terra,MODIS,65
4,36.01,37.20,2016-04-23,Aqua,MODIS,95
...,...,...,...,...,...,...
57295,42.00,26.71,2015-08-05,N,VIIRS,100
57296,42.00,26.72,2015-08-05,N,VIIRS,50
57297,42.00,41.80,2015-02-02,N,VIIRS,50
57298,42.00,42.24,2015-02-24,N,VIIRS,50


## Export
We are exporting the wildfire data with .gzip compression.

In [7]:
full_dataset.to_csv('../data/processed_data/fire_daily.csv.gz', index=False, compression='gzip')